In [1]:
#packages
import json
import tqdm
import os
import numpy as np
import sys
import glob
import pandas as pd
import datetime as dt
import re
import math
from collections import defaultdict
import operator

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins,\
ZoneVariable, HenDailyVariable_Origins, FB_process, is_day
import config_exp2 as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


In [3]:
#initialise parameter
path_initial_data = config.path_initial_data
path_extracted_data_daily_check = config.path_extracted_data_daily_check
id_run = config.id_run
dico_rc_sys = config.dico_rc_sys
path_FocalBird = config.path_FocalBird

focal_name = config.focal_name
path_Days = config.path_Days

In [4]:
#voltage thresholds
#Threshold for minimarker and wintergarden battery change
thresh_dev = 3100
#Threshold for tag battery change
thresh_tag = 2930

# Download data

In [5]:
#for the logs and device updates
over_last_Xdays = 6

### logs with their associated HenID

In [6]:
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#mtime = os.stat(os.path.join('.', p[0])).st_mtime
#download_time = dt.datetime.fromtimestamp(mtime).strftime('%Y-%m-%d-%H:%M')
#download_time = dt.datetime.strptime(download_time, '%Y-%m-%d-%H:%M')
#print('The file were downloaded at: ', download_time)
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=False, dodevice=False)
print(df.shape)
display(df.head(3))
display(df.tail(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence \T
<>:1: DeprecationWarning: invalid escape sequence \T
<ipython-input-6-dff51f11de74>:1: DeprecationWarning: invalid escape sequence \T
  p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))


C:\Users\camil\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


From the focalBirdinfo, you have 169 ative tags
(878338, 36)


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,FocalLegringName,commentSelection,weight-08-10-2021,BackpackID,commentPopulation,exception,InitialStartDate,StartDate,EndDate,ShouldBeExcluded
304809,2021-10-08 00:00:17,02002D19,tag_218,2_Zone,1.0,1.0,5.0,0.0,0.0,0.0,...,yellow5,NaN,1189.6,NaN,was selected randomly by computer as 23 but m...,,2021-09-28,2021-10-07,2022-08-01,NaN
304810,2021-10-08 00:00:19,02002D19,tag_218,3_Zone,1.0,2.0,10.0,1.0,5.0,0.0,...,yellow5,NaN,1189.6,NaN,was selected randomly by computer as 23 but m...,,2021-09-28,2021-10-07,2022-08-01,NaN
304923,2021-10-08 00:00:50,02001559,tag_59,2_Zone,1.0,1.0,6.0,0.0,0.0,0.0,...,brightgreen21,NaN,1100.6,NaN,NaN,NaN,2021-09-28,2021-10-07,2022-08-01,NaN


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,FocalLegringName,commentSelection,weight-08-10-2021,BackpackID,commentPopulation,exception,InitialStartDate,StartDate,EndDate,ShouldBeExcluded
302737,2021-11-29 09:04:41,02001E78,tag_144,3_Zone,2.0,8.0,13.0,2.0,10.0,3.0,...,pink45,NaN,1335.5,sglpt,NaN,NaN,2021-09-28,2021-10-07,2022-08-01,NaN
302738,2021-11-29 09:04:56,02001E78,tag_144,3_Zone,2.0,2.0,13.0,8.0,10.0,3.0,...,pink45,NaN,1335.5,sglpt,NaN,NaN,2021-09-28,2021-10-07,2022-08-01,NaN
304762,2021-11-29 09:04:56,02001E7C,tag_176,4_Zone,2.0,3.0,18.0,4.0,15.0,2.0,...,lightblue30,NaN,1191.2,NaN,NaN,NaN,2021-09-28,2021-10-07,2022-08-01,NaN


In [7]:
df_FB = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin') #verified dates:correct
#fill end date to today+1 for the birds which we dont know when is there end date (+1: so that today is taken into account)
df_FB['EndDate'].fillna(config.date_max+dt.timedelta(days=1), inplace=True)
df_FB['TagID'] = df_FB['TagID'].map(lambda x: 'tag_'+str(x)) #new:exp2
df_FB['PenID'] = df_FB['PenID'].map(int).map(str)
#exclude rows were tags were not functionning correctly for some reason 
df_FB = df_FB[df_FB['ShouldBeExcluded']!='yes']
#define a list with the active tags of today/last date
li_active_tags = list(df_FB[df_FB['EndDate']>=config.date_max]['TagID'].unique())
#Counter(li_active_tags)
print('From the focalBirdinfo, you have %d ative tags'%len(li_active_tags))

From the focalBirdinfo, you have 169 ative tags


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


### active tags

In [8]:
df_tag = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin')
df_tag['EndDate'].fillna(dt.date.today(), inplace=True)
print(df_tag.shape)
df_tag = df_tag[df_tag['ShouldBeExcluded']!='yes']
df_tag['TagID'] = df_tag['TagID'].map(lambda x: 'tag_'+str(x)) #new:exp2
print(df_tag.shape)
df_tag.head(3)

(169, 18)
(169, 18)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


,HenID,SireID,RPenID,PenID,Rtreatment,LPLegring,legringID,FocalLegringName,commentSelection,weight-08-10-2021,BackpackID,TagID,commentPopulation,exception,InitialStartDate,StartDate,EndDate,ShouldBeExcluded
0,1,15,Rpen12,10,Queue,red,1,red1,was slected as 46 by random on computer but n...,1122.0,NaN,tag_5,NaN,NaN,28.09.2021,2021-10-07,2022-08-01,NaN
1,2,63,Rpen13,10,NoQueue,red,17,red17,NaN,1343.8,NaN,tag_16,NaN,NaN,28.09.2021,2021-10-07,2022-08-01,NaN
2,3,78,Rpen13,10,NoQueue,red,15,red15,NaN,1252.7,NaN,tag_19,NaN,NaN,28.09.2021,2021-10-07,2022-08-01,NaN


In [9]:
li_active_tags = list(set(df_tag[df_tag['EndDate']>=pd.to_datetime(dt.date.today())]['TagID'].unique()))
#Counter(li_active_tags)
print('From your focalbird document you should have %d ative tags'%len(li_active_tags))

From your focalbird document you should have 169 ative tags


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### device

In [10]:
#select last x logs
last_x_logs = math.ceil(over_last_Xdays/2) #arondi en haut
li_logs = []
for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
    li_ = glob.glob(os.path.join(path_system, 'log*'))
    li_.sort(key=lambda x: os.path.getmtime(x), reverse=False)
    li_logs.extend(li_[-last_x_logs:]) 
#next version
#li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
#                     'LFCounter','LFRSSISum','Zone', 'Subzone','LFRSSI','Time','LastInfo','LastLocation','LastAction']
li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
                     'LFCounter','LFRSSISum','Zone', 'Subzone','Time','LastInfo','LastLocation','LastAction']
#lastReg = LastAction
li_df = []
for path_log in tqdm.tqdm(li_logs):
    #confused month and day
    df_device = pd.read_csv(os.path.join(path_log), sep=';', names=li_newFirmwarecol, parse_dates=['LastAction','Timestamp'],
                           dayfirst=True)
    df_device['data_path'] = path_log
    df_device['system'] = 'pens:'+path_log.split('Barn 4 Pen ')[1].split('\\')[0]
    df_device['logID'] = path_log.split('\\')[-1].split('.')[0]
    #df_device = df_device[:-1] #remove last row as str some times 
    #df_device['Timestamp'] = df_test['Timestamp'].map(lambda x: dt.datetime.strptime(x, "%d.%m.%Y %H:%M:%S")) #faster than parse_dates
    df_device['date'] = df_device['Timestamp'].map(lambda x: dt.datetime.date(x))
    print(df_device.shape)
    print('-----------------', path_log)
    #display(df_device.head(3))
    #display(df_device.tail(3))
    li_df.append(df_device)
df_device = pd.concat(li_df)
df_device = df_device[df_device['Timestamp']>=dt.datetime(2020,9,29)]
df_device['sender'].fillna(' ', inplace=True)
#add the info on if the device needs batterie or not
df_device['NeedsBatteries'] = df_device.apply(lambda x: ('mini' in x['sender']) | (x['sender']=='Wintergarten') | \
                                              (x['sort']=='Tag'), axis=1)

#add 'tag_' to the tag names
df_device['sender_tag'] = df_device['sender'].map(lambda x: 'tag_'+str(x)) #new:exp2
df_device['sender'] = np.where(df_device['sort']=='Tag', df_device['sender_tag'], df_device['sender'])

#remove old tags
print(df_device.shape)
df_device = df_device[~((df_device['sort']=='Tag')&(~df_device['sender'].isin(li_active_tags)))]
print(df_device['logID'].unique())
print(df_device.shape)
df_device.tail(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence \D
<>:4: DeprecationWarning: invalid escape sequence \D
<ipython-input-10-7fe44d0a0414>:4: DeprecationWarning: invalid escape sequence \D
  for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
 11%|█████████▎                                                                          | 1/9 [00:19<02:33, 19.15s/it]

(216216, 20)
----------------- G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\GantnerSystem\DATA\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000025.csv


 22%|██████████████████▋                                                                 | 2/9 [00:32<02:00, 17.27s/it]

(254208, 20)
----------------- G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\GantnerSystem\DATA\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000024.csv


 33%|████████████████████████████                                                        | 3/9 [00:47<01:39, 16.60s/it]

...

(381636, 20)
----------------- G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\GantnerSystem\DATA\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000016.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:01<00:00, 13.46s/it]

(153650, 20)
----------------- G:\VPHI\Welfare\2- Research Projects\Camille Montalcini\Origins.GS\GantnerSystem\DATA\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000018.csv


(2108376, 22)
['log_00000025' 'log_00000024' 'log_00000026' 'log_00000023'
 'log_00000017' 'log_00000016' 'log_00000018']
(2048937, 22)


,Timestamp,sort,code,sender,Temperature,Battery Voltage,MovementCounter,LastLFSeen,LFCounter,LFRSSISum,...,Time,LastInfo,LastLocation,LastAction,data_path,system,logID,date,NeedsBatteries,sender_tag
153647,2021-11-29 09:02:12,Tag,02002178,tag_191,28.0,3006.0,49.0,0.0,810.0,4781.0,...,124.0,29.11.2021 08:55:15,29.11.2021 08:53:10,2021-11-29 09:01:17,G:\VPHI\Welfare\2- Research Projects\Camille M...,pens:8 - 9,log_00000018,2021-11-29,True,tag_191
153648,2021-11-29 09:02:12,Tag,020026EF,tag_205,28.0,2995.0,52.0,0.0,776.0,9937.0,...,255.0,29.11.2021 09:01:20,29.11.2021 06:53:28,2021-11-29 09:01:51,G:\VPHI\Welfare\2- Research Projects\Camille M...,pens:8 - 9,log_00000018,2021-11-29,True,tag_205
153649,2021-11-29 09:02:12,Tag,020026FF,tag_207,26.0,3012.0,54.0,1.0,514.0,5972.0,...,255.0,29.11.2021 08:52:09,29.11.2021 05:36:35,2021-11-29 09:00:17,G:\VPHI\Welfare\2- Research Projects\Camille M...,pens:8 - 9,log_00000018,2021-11-29,True,tag_207


In [11]:
#df_device.groupby(['system','logID'])['NeedsBatteries'].count().reset_index()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#df_device[df_device['logID']=='log_00000010']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#an error might be cause if the alst row was not complet when the file was downloaded

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
#df_device[df_device['sort']=='Marker'][['code','sender','system']].drop_duplicates() #all actives at least once

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
#df_device[(df_device['sort']=='Tag')&(df_device['Battery Voltage']<0)]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Use only the last few days

In [18]:
download_time = max(df_device['Timestamp'].tolist())

#remove the days we dont want
df_device['to_analyse'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df_device = df_device[df_device['to_analyse']]

path_daily_ver = os.path.join(path_extracted_data_daily_check,str(download_time).split(' ')[0])
#create a director if not existing
if not os.path.exists(path_daily_ver):
    os.makedirs(path_daily_ver)

#keep only the last over_last_Xdays days in the logs
df['to_analyse'] = df['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df = df[df['to_analyse']]
df[df['to_analyse']]['date'].unique()    

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['2021-11-23T00:00:00.000000000', '2021-11-24T00:00:00.000000000',
       '2021-11-25T00:00:00.000000000', '2021-11-26T00:00:00.000000000',
       '2021-11-27T00:00:00.000000000', '2021-11-28T00:00:00.000000000',
       '2021-11-29T00:00:00.000000000'], dtype='datetime64[ns]')

In [19]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-11-29 09:02:12')

# Tags last records

In [20]:
#tags last records and active tags with no records
df_ = df[df['TagID'].isin(li_active_tags)].groupby(['TagID','PenID'])['Timestamp'].agg(lambda x: list(x)).reset_index()
df_['last_record_timestamp'] = df_['Timestamp'].map(lambda x: max(x))
df_['nbr_h_since_lastrecord'] = df_['last_record_timestamp'].map(lambda x: int((download_time-x).total_seconds()/60/60))

#add active tags that had no transition at all
li_no_transition = [i for i in li_active_tags if i not in df['TagID'].unique()]
df_no_transition = pd.DataFrame([{'TagID':tag_, 'nbr_h_since_lastrecord':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
#remove the older row of tags taht appear two times (i.e. if they changed pens for example)
df_.sort_values(['nbr_h_since_lastrecord'], ascending=False, inplace=True)
df_.drop_duplicates(subset=['TagID'], keep='last', inplace=True)
df_[['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].to_csv(os.path.join(path_daily_ver, 
                                                                           id_run+str(download_time).split(' ')[0]+'h_TagsLastRecords.csv'), 
                                                              sep=';', index=False)

h = 10
print('------------- Tags with no records since at least %d hours: -------------'%h)
print('There is %d such tags, we will print a maximum of 50 line (starting form the worst ones)'%df_[df_['nbr_h_since_lastrecord']>h].shape[0])
li_tag_issues = df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']]['TagID'].tolist()
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))
df_tag_summary = df_.copy()
df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].head(50)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1, 2) (168, 5)
(169, 5)
------------- Tags with no records since at least 10 hours: -------------
There is 2 such tags, we will print a maximum of 50 line (starting form the worst ones)
There is 2 tags with at least one issue


,TagID,PenID,last_record_timestamp,nbr_h_since_lastrecord
0,tag_89,NaN,NaT,9999
133,tag_52,pen11,2021-11-28 16:42:32,16


In [21]:
#df[df['TagID']=='19'] #10.04 morning: verify in video
#df_device[df_device['sender']=='29']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Reader last update

#understand how the device logs are
df_test = df_device.groupby('code')['system'].agg(lambda x: len(set(x))).reset_index()
df_test[df_test['system']>1] #--> no code in two system!!
#importantly:
df_device[df_device['sort']!='Tag'].groupby(['sender'])['system','code','sort'].agg(lambda x: set(x)).reset_index()

In [22]:
for reader_code in dico_rc_sys.keys():
    print("The last read out from the reader of %s arrived at: %s"%(dico_rc_sys[reader_code],
                                                    str(max(df_device[df_device['code']==reader_code]['LastAction'].tolist()))))

The last read out from the reader of Reader Pen 3-5 arrived at: 2021-11-29 08:52:15
The last read out from the reader of Reader Pen 8-9 arrived at: 2021-11-29 09:02:08
The last read out from the reader of Reader Pen 10-12 arrived at: 2021-11-29 08:56:56


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#cleaning step: remove days for specific hens when the tag was not sending output regularly anymore
df_test = df_device[df_device['sort']=='Tag'].copy()
df_test['date'] = df_test['Timestamp'].map(lambda x: dt.datetime.date(x))
df_test = df_test.groupby(['sender','date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_test.rename(columns={'sender':'TagID'}, inplace=True)
df_test['nbr_last_action']
#last action biggest gap
print(df_test.shape)
df_test.head(3)

## LF Counter

In [23]:
df_LFCoutner = df_device[df_device['sort']=='Tag'].copy()
df_LFCoutner['date'] = df_LFCoutner['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_LFCoutner['is_day'] = df_LFCoutner['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
df_LFCoutner = df_LFCoutner.groupby(['sender','date'])['LFCounter'].agg(lambda x: sorted(list(x))).reset_index()
df_LFCoutner['LFCounter_nbr_equal0'] = df_LFCoutner['LFCounter'].map(lambda x: sum([i==0 for i in x]))
df_LFCoutner['LFCounter_atleastone0'] = df_LFCoutner['LFCounter_nbr_equal0'].map(lambda x: x>0)
print(df_LFCoutner.shape)
df_ = df_LFCoutner[df_LFCoutner['LFCounter_atleastone0']].groupby(['sender'])[['date',
                                                                'LFCounter_nbr_equal0']].agg(lambda x: list(x)).reset_index()
df_.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LFCounterEqual0.csv'),sep=';')
df_

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1183, 5)


,sender,date,LFCounter_nbr_equal0
0,tag_107,"[2021-11-23, 2021-11-24, 2021-11-26, 2021-11-2...","[2, 1, 2, 4, 1]"
1,tag_108,[2021-11-24],[1]
2,tag_119,[2021-11-24],[1]
3,tag_135,[2021-11-24],[1]
4,tag_16,"[2021-11-23, 2021-11-25, 2021-11-26, 2021-11-27]","[1, 1, 1, 1]"
5,tag_167,[2021-11-28],[1]
6,tag_19,"[2021-11-23, 2021-11-24, 2021-11-27]","[1, 1, 2]"
7,tag_203,[2021-11-23],[1]
8,tag_25,[2021-11-26],[1]
9,tag_32,"[2021-11-25, 2021-11-27, 2021-11-28]","[1, 1, 1]"


# Device Last update

In [24]:
#TODO
#New data!!!: plt.hist(df_device['MovementCounter'].dropna(),bins='auto');
#plt.hist(df_device['Temperature'].dropna(),bins=30);

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## biggest gap over the entire day

In [25]:
df_lastactionGAP = df_device[df_device['sort']=='Tag'].copy()
df_lastactionGAP['date'] = df_lastactionGAP['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_lastactionGAP['is_day'] = df_lastactionGAP['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
#df_lastactionGAP = df_lastactionGAP[df_lastactionGAP['is_day']]
df_lastactionGAP = df_lastactionGAP.groupby(['sender','date'])['LastAction'].agg(lambda x: sorted(list(x))).reset_index()
#last action biggest gap of the day
df_lastactionGAP['li_gap_of_the_day_tsstarted'] = df_lastactionGAP['LastAction'].map(lambda x: {x[i]:(x[i+1]-x[i]).total_seconds() for i in range(0,len(x)-1)})
df_lastactionGAP['biggest_gap_of_day_mn'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: round(max(x.items(), 
                                                                                         key=operator.itemgetter(1))[1]/60,0))
df_lastactionGAP['start_biggest_gap_of_day'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: max(x.items(), 
                                                                                               key=operator.itemgetter(1))[0])
df_lastactionGAP.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LastactionGAP.csv'),sep=';')
print(df_lastactionGAP.shape)
df_lastactionGAP[df_lastactionGAP['biggest_gap_of_day_mn']>12]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1183, 6)


,sender,date,LastAction,li_gap_of_the_day_tsstarted,biggest_gap_of_day_mn,start_biggest_gap_of_day
750,tag_219,2021-11-24,"[2021-11-24 00:01:20, 2021-11-24 00:11:23, 202...","{2021-11-24 00:01:20: 603.0, 2021-11-24 00:11:...",14.0,2021-11-24 18:18:05
753,tag_219,2021-11-27,"[2021-11-27 00:01:21, 2021-11-27 00:11:34, 202...","{2021-11-27 00:01:21: 613.0, 2021-11-27 00:11:...",22.0,2021-11-27 18:39:21
754,tag_219,2021-11-28,"[2021-11-28 00:02:14, 2021-11-28 00:12:19, 202...","{2021-11-28 00:02:14: 605.0, 2021-11-28 00:12:...",22.0,2021-11-28 23:19:53
1099,tag_83,2021-11-23,"[2021-11-23 09:04:27, 2021-11-23 09:14:34, 202...","{2021-11-23 09:04:27: 607.0, 2021-11-23 09:14:...",33.0,2021-11-23 11:52:07
1101,tag_83,2021-11-25,"[2021-11-25 00:04:53, 2021-11-25 00:14:58, 202...","{2021-11-25 00:04:53: 605.0, 2021-11-25 00:14:...",286.0,2021-11-25 16:58:39
1102,tag_83,2021-11-26,"[2021-11-25 21:47:05, 2021-11-25 21:47:05, 202...","{2021-11-25 21:47:05: 19666.0, 2021-11-26 03:1...",328.0,2021-11-25 21:47:05
1103,tag_83,2021-11-27,"[2021-11-26 16:35:07, 2021-11-26 16:35:07, 202...","{2021-11-26 16:35:07: 49249.0, 2021-11-27 06:1...",821.0,2021-11-26 16:35:07
1104,tag_83,2021-11-28,"[2021-11-28 00:04:38, 2021-11-28 00:14:42, 202...","{2021-11-28 00:04:38: 604.0, 2021-11-28 00:14:...",200.0,2021-11-28 11:16:13


### very last update

In [26]:
#sort by date in order to take the last voltage using a simple list
df_ = df_device.groupby(['code','sender','system','sort','NeedsBatteries'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_['last_LastAction'] = df_['LastAction'].map(lambda x: max(x))
df_['nbr_mn_since_LastUpdate'] = df_['last_LastAction'].map(lambda x: (download_time-x).total_seconds()/60)

#add tags with no update:
li_no_transition = [i for i in li_active_tags if i not in df_device['sender'].unique()]
df_no_transition = pd.DataFrame([{'sender':tag_, 'nbr_mn_since_LastUpdate':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
print('There is %d device with 0 updates'%df_no_transition.shape[0])
df_.sort_values(['nbr_mn_since_LastUpdate'], ascending=False, inplace=True)
#remove the older sender that appear two times (i.e. if they changed pens for example)
df_.drop_duplicates(subset=['system','sender'], keep='last', inplace=True)
df_[['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']].to_csv(os.path.join(path_daily_ver, 
                                            id_run+str(download_time).split(' ')[0]+'h_DeviceLastUpdate.csv'), sep=';', index=False)


mn = 12
li_tag_issues.extend(df_[df_['nbr_mn_since_LastUpdate']>mn]['sender'])
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))

print('------------- Device with no update since at least %d minutes -------------'%mn)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'], left_on='TagID', right_on='sender', how='outer')
#display(df_tag_summary.head(3))
display(df_[df_['nbr_mn_since_LastUpdate']>mn][['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']])

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, 0) (196, 8)
(196, 8)
There is 0 device with 0 updates
There is 2 tags with at least one issue
------------- Device with no update since at least 12 minutes -------------


,code,sender,system,sort,last_LastAction,nbr_mn_since_LastUpdate


In [27]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-11-29 09:02:12')

# Critical Voltage last D days

In [28]:
D = 7 #assuming I do this one a week, it make sense to plot it once a week
#D=2 was not enough as we would miss some tags that were at 2600 (e.g. 82)
df_device['is_last_'+str(D)+'days'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=D)))
df_ = df_device[df_device['is_last_'+str(D)+'days']].groupby(['code','sender','system','sort',
                                                        'NeedsBatteries'])['Battery Voltage'].agg(lambda x: list(x)).reset_index()
df_['min_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: min(x))
df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))
df_.sort_values(['min_Battery_Voltage_last_'+str(D)+'days'], ascending=False, inplace=True)
# check whether at least one of the last entries was below the critical threshold 
df_['CRITICAL'] = df_.apply(lambda x:sum([i<thresh_dev for i in x['Battery Voltage']])>=1 if x['sort']!='Tag' \
                            else sum([i<thresh_tag for i in x['Battery Voltage']])>=1, axis=1)
df_.to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'h_Battery VoltageLast'+str(D)+'days.csv'), sep=';', index=False)
print('------------- Critical Battery Voltage of device/tags (under threshold at least once during the last %d days -------------'%D)
li_tag_issues.extend(df_[(df_['CRITICAL'])&(df_['sort']=='Tag')]['sender'])
print(df_tag_summary.shape)
df_tag_summary = df_tag_summary.drop(['sender'], axis=1)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'][['CRITICAL','min_Battery_Voltage_last_'+str(D)+'days',
                                                                   'avg_Battery_Voltage_last_'+str(D)+'days','sender']], 
                          left_on='TagID', right_on='sender', how='outer')
df_tag_summary.drop(['LastAction','Timestamp','NeedsBatteries'], axis=1).to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'_TagSummary.csv'), sep=';', index=False)
print(df_tag_summary.shape)
#display(df_tag_summary.head(3))
print('There is %d tags with at least one issue:'%len(set(li_tag_issues)))
df_[(df_['CRITICAL'])&(df_['NeedsBatteries'])]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-28-207c442b845e>:7: RuntimeWarning: Mean of empty slice
  df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))


------------- Critical Battery Voltage of device/tags (under threshold at least once during the last 7 days -------------
(169, 13)
(169, 16)
There is 13 tags with at least one issue:


,code,sender,system,sort,NeedsBatteries,Battery Voltage,min_Battery_Voltage_last_7days,avg_Battery_Voltage_last_7days,CRITICAL
116,02001E67,tag_167,pens:8 - 9,Tag,True,"[3008.0, 3003.0, 3005.0, 3002.0, 3004.0, 3003....",2921.0,3003.366474,True
63,020015BC,tag_16,pens:10 - 12,Tag,True,"[3009.0, 3011.0, 3009.0, 3011.0, 3006.0, 3007....",2918.0,3007.430058,True
77,0200196D,tag_119,pens:3 - 5,Tag,True,"[3006.0, 3003.0, 3007.0, 3002.0, 3002.0, 3004....",2910.0,3001.432870,True
29,0200152C,tag_42,pens:8 - 9,Tag,True,"[2999.0, 2991.0, 2992.0, 2991.0, 2995.0, 2991....",2909.0,2989.986127,True
120,02001E6C,tag_168,pens:8 - 9,Tag,True,"[2937.0, 2937.0, 2937.0, 2938.0, 2937.0, 2933....",2903.0,2929.661272,True
14,02001419,tag_19,pens:10 - 12,Tag,True,"[3026.0, 3029.0, 3023.0, 3025.0, 3026.0, 3026....",2902.0,3024.166474,True
32,02001530,tag_107,pens:8 - 9,Tag,True,"[2994.0, 2996.0, 2998.0, 2998.0, 2995.0, 2997....",2899.0,2994.021965,True
70,020015CA,tag_89,pens:8 - 9,Tag,True,"[2950.0, 2949.0, 2944.0, 2945.0, 2947.0, 2945....",2891.0,2978.541040,True
28,02001525,tag_25,pens:3 - 5,Tag,True,"[2999.0, 2999.0, 2999.0, 3003.0, 3001.0, 2999....",2888.0,2999.018519,True
96,02001E26,tag_170,pens:3 - 5,Tag,True,"[2844.0, 2854.0, 2843.0, 2826.0, 2834.0, 2850....",2763.0,2838.350694,True


#plot

df_device = df_device.sort_values(['LastAction'])
for tag in li_active_tags:
    df_plt = df_device[df_device['sender']==str(tag)][['voltage','Timestamp']]
    plt.plot(df_plt['Timestamp'].tolist(), df_plt['voltage'].tolist());
    plt.title(tag)
    plt.show()

# Plot Tags

#plot the batterie level, the tagupdates and the number of transition for each tags, one plot per day
df_device['date'] = df_device['Timestamp'].map(lambda x: dt.datetime.date(x))
df_device = df_device.sort_values(['LastAction'])
#us the log files to crate a df_tr file with one row per hen per hour, and the number of transition in a column
df['hour'] = df['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
df_tr = df.groupby(['TagID','date','hour'])['Timestamp'].count().reset_index()
df_tr.rename(columns={'Timestamp':'nbr_transition_last1h'}, inplace=True)
df_tr['date_hour'] = df_tr.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)

#for tag in tqdm.tqdm(set(li_tag_issues)):
for tag in tqdm.tqdm(li_active_tags):
    try:
        plt.figure(figsize=(18,4))
        ax = plt.subplot(111) 

        #Battery Voltage
        df_plt = df_device[df_device['sender']==tag].copy()
        SystemID = df_plt['system'].iloc[0]
        ax.plot(df_plt['Timestamp'].tolist(), df_plt['Battery Voltage'].tolist(), color='blue');

        #number of transition in the last hours    
        ax2=ax.twinx()
        df_tr_ = df_tr[df_tr['TagID']==tag].copy()
        ax2.plot(df_tr_['date_hour'].tolist(), df_tr_['nbr_transition_last1h'].tolist(), color='green');

        #number of device update in the last hour
        df_plt['hour'] = df_plt['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
        df_up = df_plt.groupby(['date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
        df_up['date_hour'] = df_up.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)
        df_up['nbr_update_last1h'] = df_up.apply(lambda x: sum([i>=(x['date_hour']-dt.timedelta(hours=1)) for i in x['LastAction']]), 
                                                 axis=1)
        ax2.plot(df_up['date_hour'].tolist(), df_up['nbr_update_last1h'].tolist(), color='r');

        #add days and plot
        for day in df_plt['date'].unique() : 
            plt.axvline(x=day, linewidth=1, color='b')
        plt.title('TagID: '+tag)
        #for label in ax.get_xticklabels():
        #    label.set_rotation(90) 
        y_ = np.nanmax(df_plt['Battery Voltage'].tolist())
        x_ = np.nanmin(df_plt['date'].unique())
        ax.text(x_, y_-0.1*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#device update in the last hour -right axis', fontsize=10, color='r')
        ax.text(x_, y_-0.2*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#transition in the last hour -right axis', fontsize=10, color='green')
        ax.text(x_, y_-0.3*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), 'Battery Voltage -left axis', fontsize=10, color='blue')

        #create a director if not existing
        path_incase = os.path.join(path_daily_ver, 'INcaseALLplot', SystemID.replace(':',''))
        if not os.path.exists(path_incase):
            os.makedirs(path_incase)
        plt.savefig(os.path.join(path_incase,id_run+'_'+str(tag)+'Tag_CHECK.png'), dpi=300,
                                format='png', bbox_inches='tight')
    except Exception as e:
                print('ERROR:-------------',e, '    TAG: ',tag)
    #plt.show()